In [6]:
import requests
import pandas as pd
import json
import numpy as np
import time
import datetime

In [9]:
token = 'c93bb43058433337f34acdc63ac9e42d1cf53ac6'
api_base = 'https://www.renewables.ninja/api/'

def get_ninja_solar_data(session, lat, lon, year):
    url = api_base + 'data/pv'
    args = {
        'lat': lat,
        'lon': lon,
        'date_from': f'{year}-01-01',
        'date_to': f'{year}-12-31',
        'dataset': 'merra2',
        'capacity': 1.0,
        'system_loss': 0.1,
        'tracking': 1,
        'tilt': 35,
        'azim': 180,
        'format': 'json'
    }

    r = session.get(url, params=args)
    
    # Parse JSON to get a pandas.DataFrame of data and dict of metadata
    parsed_response = json.loads(r.text)
    
    data = pd.read_json(json.dumps(parsed_response['data']), orient='index')
    data["electricity"] = np.roll(data["electricity"].values, -6)

    metadata = parsed_response['metadata']

    return data, metadata

def get_ninja_wind_data(session, lat, lon, year):
    url = api_base + 'data/wind'
    args = {
        'lat': lat,
        'lon': lon,
        'date_from': f'{year}-01-01',
        'date_to': f'{year}-12-31',
        'capacity': 1.0,
        'height': 90,
        'turbine': 'Vestas V110 2000',
        'format': 'json'
    }

    r = session.get(url, params=args)
    
    # Parse JSON to get a pandas.DataFrame of data and dict of metadata
    parsed_response = json.loads(r.text)
    
    data = pd.read_json(json.dumps(parsed_response['data']), orient='index')
    data["electricity"] = np.roll(data["electricity"].values, -6)

    metadata = parsed_response['metadata']

    return data, metadata


In [ ]:

s = requests.session()
# Send token header with each request
s.headers = {'Authorization': 'Token ' + token}


##
# PV example
##
lat_lon = [(31.1445, -103.269), (32.294, -101.9287), (31.707, -100.3027), (30.3826, -97.5175), (29.7096, -95.7733)]
site_ids = range(7, 12)
years = range(2002, 2023)
url = api_base + 'data/pv'

data_list = []
for year in range(2003, 2023):
    year_list = []
    print(year)
    for site_id, (lat, lon) in zip(site_ids, lat_lon):
        # print(site_id)
        _data, _ = get_ninja_solar_data(s, lat, lon, year)
        _data.columns = [site_id]
        year_list.append(_data)
        time.sleep(20)
    year_data = pd.concat(year_list, axis=1)
    try:
        year_data.index.name = "local_time"
    except:
        print("setting index name didn't work")
    year_data.to_csv(f"{year}_renewables_ninja_solar_profiles.csv")
    print(time.time())


In [12]:

s = requests.session()
# Send token header with each request
s.headers = {'Authorization': 'Token ' + token}


##
# Wind
##
lat_lon = [(32.7479, -100.6653), (31.8937, -101.3904), (31.7163, -100.3247), (28.4372, -96.6003), (27.6128, -97.5891), (26.4876, -97.655)]
site_ids = range(1, 7)
# years = range(2002, 2023)
# url = api_base + 'data/pv'

data_list = []
for year in range(2002, 2023):
    year_list = []
    print(year)
    for site_id, (lat, lon) in zip(site_ids, lat_lon):
        # print(site_id)
        _data, _ = get_ninja_wind_data(s, lat, lon, year)
        _data.columns = [site_id]
        year_list.append(_data)
        time.sleep(20)
    year_data = pd.concat(year_list, axis=1)
    try:
        year_data.index.name = "local_time"
    except:
        print("setting index name didn't work")
    year_data.to_csv(f"{year}_renewables_ninja_wind_profiles.csv")
    print(datetime.datetime.now().strftime("%H:%M:%S"))


2002
15:12:46
2003
15:14:55
2004
15:17:04
2005
15:19:13
2006
15:21:22
2007
15:23:31
2008
15:25:41
2009
15:27:50
2010
15:29:59
2011
15:32:09
2012
15:34:17
2013
15:36:27
2014
15:38:36
2015
15:40:46
2016
15:42:55
2017
15:45:04
2018
15:47:13
2019
15:49:22
2020
15:51:32
2021
15:53:41
2022
15:55:50
